In [1]:
import os
import numpy as np
import pandas as pd

DATA_DIR="data/"

In [2]:
cats=pd.read_csv("data/cat_levels.csv")
cats.head()

,level1_categories,level_2_categories
0,Info Tech,Info Tech
1,Travel & Tourism,PaaS
2,Travel,Products
3,Vehicles & Parts,SaaS
4,Automobiles,Travel & Tourism


In [4]:
sample=pd.read_csv("data/TIL_sample_submission.csv")
sample

,id,category_tree
0,21047,entertainment->bollywood
1,21047,News & Politics -> Politics -> bjp -> dharmend...
2,243958,sports -> Outdoor Games - > Tennis -> novak dj...
3,243958,location -> serbia


In [2]:
with open(DATA_DIR+"Train_data.csv", "r") as f:
    print(f.readline().strip("\n"))


title|link|description|long_description|id


In [3]:
import re


#checks=["'", '"']
checks=['"']
def unbalanced(s):
    for check in checks:
        if (s.count(check)-s.count(f'\\{check[0]}'))%2!=0:
            return True
    return False
def lineCleaner(line):
    line=line.strip("\n")
    splitl=line.split("|")
    slen=len(splitl)
    ids=re.findall(r'\|([0-9]+)$',line)
    if slen==5:
        return splitl
    else:
        try:
            lindex=0
            res=[]
            while(lindex<slen):
                if unbalanced(splitl[lindex]) and lindex<slen-1:
                    rindex=lindex+1
                    while not unbalanced(splitl[rindex]) and rindex<slen-2:
                        rindex+=1
                    res+=[" ".join(splitl[lindex:rindex+1])]
                    lindex=rindex
                else:
                    res+=[splitl[lindex]]
                lindex+=1
#             if len(res)==6:
#                 res[3]=res[3]+res[4]
#                 res[]
            if len(res)>5:
                res[3]=" ".join(res[3:])
                res=res[:4]
            if len(res)==4:
                res+=[ids[0].strip("|")]
#             if len(res)==3:
#                 long_desc=re.search(r'\|([^\|]*)\|[0-9]*$', line)
            if len(res)!=5:
                res=[]
                url=re.search(r'\|http.*?\|', line)
                if url is not None:
                    res+=[line[:url.start()]]
                    res+=[line[url.start():url.end()]]
                else:
                    res+=[None, None]
                long_desc=re.search(r'\|([^\|]*)\|[0-9]*$', line)
                if long_desc is not None:
                    res+=[line[url.end():long_desc.start()]]
                    res+=[line[long_desc.start():long_desc.end()]]
                else:
                    res+=[None, None]
                res+=[ids[0]]
                for i in range(5):
                    if res[i] is not None:
                        res[i]=res[i].strip("|")
            return res
        except Exception as e:
            return [None]*5
#             res=[ids[0].strip("|")]
#             url=re.search(r'\|http.*?\|', line).start()
            
#             long_desc=re.search(r'\|([^\|]*)\|[0-9]*$', line)

In [126]:
# import re


# #checks=["'", '"']
# checks=['"']
# def unbalanced(s):
#     for check in checks:
#         if (s.count(check)-s.count(f'\\{check[0]}'))%2!=0:
#             return True
#     return False
# def lineCleaner(line):
#     line=line.strip("\n")
#     splitl=line.split("|")
#     slen=len(splitl)
#     ids=re.findall(r'\|([0-9]+)$',line)
#     if slen==5:
#         return splitl
#     else:
#         try:
#             res=[]
#             url=re.search(r'\|http.*?\|', line)
#             if url is not None:
#                 res+=[line[:url.start()]]
#                 res+=[line[url.start():url.end()]]
#             else:
#                 res+=[None, None]
#             long_desc=re.search(r'\|([^\|]*)\|[0-9]*$', line)
#             if long_desc is not None:
#                 res+=[line[url.end():long_desc.start()]]
#                 res+=[line[long_desc.start():long_desc.end()]]
#             else:
#                 res+=[None, None]
#             res+=[ids[0]]
#             for i in range(5):
#                 if res[i] is not None:
#                     res[i]=res[i].strip("|")
#             return res
#         except Exception as e:
#             return [None]*5
# #             res=[ids[0].strip("|")]
# #             url=re.search(r'\|http.*?\|', line).start()
            
# #             long_desc=re.search(r'\|([^\|]*)\|[0-9]*$', line)

In [9]:
count=0
with open(DATA_DIR+"Train_data.csv", "r") as f:
    for line in tqdm(f, total=1412721):
        if line.find(u'\u0001')!=-1:
            count+=1
print(count)


0


In [ ]:
count=0
badcount=0
with open(DATA_DIR+"Train_data.csv", "r") as f:
    for line in f:
        if len(line.split("|"))>5:
            try:
                if len(line[re.search(r'\|http.*?\|', line).start():].split("|"))>5:
                    count+=1
                    if count<10:
                        print(line)
            except:
                count+=1

In [16]:
count

1412721

In [137]:
badcount

78

In [168]:
from csv import reader
count=0
with open(DATA_DIR+"Train_data.csv", "r") as f:
    feader=f.readline().strip("\n")
    rc=reader(f, delimiter="|", quotechar='"')
    for row in tqdm(rc, total=1411103):
        try:
            int(row[-1])
        except Exception as e:
            count+=1
            print(row)

In [ ]:
from io import StringIO
from csv import writer, reader 
import pandas as pd
from tqdm.auto import tqdm

output = StringIO()
csv_writer = writer(output, delimiter=u'\u0001')

bad_rows=[]

with open(DATA_DIR+"Train_data.csv", "r") as f:
    header=f.readline().strip("\n")
    csv_writer.writerow(header.split("|"))
    rc=reader(f, delimiter="|", quotechar='"')
    for row in tqdm(rc, total=1411103):
        if len(row)!=5:
            try:
                row_new=lineCleaner("|".join(row))
                int(row_new[-1])
            except Exception as e:
                bad_rows+=[row]
                row_new=[None, None, None, None, row[-1]]
            finally:
                row=row_new
        csv_writer.writerow(row)
        

output.seek(0)
df_first = pd.read_csv(output, sep=u'\u0001')
df_first.head()


In [17]:
badcount

71088

In [ ]:
cats=pd.read_csv(DATA_DIR+"cat_levels.csv")
sample=pd.read_csv(DATA_DIR+"TIL_sample_submission.csv")
df=pd.read_csv(DATA_DIR+"Train_data.csv")

In [4]:
sample

,id,category_tree
0,21047,entertainment->bollywood
1,21047,News & Politics -> Politics -> bjp -> dharmend...
2,243958,sports -> Outdoor Games - > Tennis -> novak dj...
3,243958,location -> serbia


In [ ]:
df_train['link'].to_csv(DATA_DIR+"links.csv", sep=u'\u0001')